In [1]:
import os
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [2]:
import os

dir_path = os.path.join('.', 'auto_mining')  # starting dir

completed_problem = 0
problems = []
difficulty = []
status = []
passCount = []
totalCount = []
description = []

for folder_name in os.listdir(dir_path):
    result_dir = os.path.join(dir_path, folder_name, 'result')
    # the status needs to be extracted from the last line of the 1.txt files in the result
    # folder: Run[31m Time Error (0.26�s)[0m
    # it might be worth it to just print out all the last line to confirm patterns,
    #
    status_dir = os.path.join(result_dir, '2.chatgpt.txt')

    # Check if 1.txt exists
    if not os.path.exists(status_dir):
        print(f"File 2.chatgpt.txt does not exist for problem {folder_name}. Skipping.")
        continue

    # Open the file and read the first line
    with open(status_dir, 'r', encoding='utf-8') as status_file:
        lines = status_file.readlines()
        first_line = lines[0].strip()
        # print(lines[-4].strip())
        # Check if the first line indicates submission token issue
        if first_line.startswith("You are out of submission tokens"):
            print(f"Problem {folder_name} has submission token issue. Skipping.")
            continue  # Move to the next problem if submission token issue

        # Read all lines and get the last line
        
        if len(lines) >= 3:
            # the number of passed tests to be extracted from the second to last line, as well as
            # the total number of tests
            second_to_last_line = lines[-4].strip()
            # print(second_to_last_line)
            numbers = re.findall(r'\d+', second_to_last_line)
            if len(numbers) >= 2:
                status_line = lines[-2].strip()  # Get the last line
                start_index = status_line.find('m')  # Find the index of the first "m"
                end_index = status_line.find('(')  # Find the index of "("
                if start_index != -1 and end_index != -1:
                    middle_part = status_line[start_index+1:end_index].strip()  # Extract the middle part
                    status.append(middle_part)  # Append the middle part to the status list
                else:
                    print("Start or end delimiter not found in the status line")
                # status.append(lines[-2].strip())  # type of text
                passCount.append(int(numbers[-2]))
                totalCount.append(int(numbers[-1]))
            else:
                print("Could not extract numbers from the line")
                continue
        else:
            print(f"File 2.chatgpt.txt for problem {folder_name} is empty. Skipping.")
            continue

    if os.path.isdir(result_dir):
        metadata_file_path = os.path.join(dir_path, folder_name, 'metadata')  # Path to the metadata file
        # Open the metadata file and read the third line
        with open(metadata_file_path, 'r') as metadata_file:
            # Read the third line (indexing starts from 0)
            lines = metadata_file.readlines()
            if len(lines) >= 3:  # Check if the file has at least 3 lines               
                third_line = lines[2]  # Third line of the file
                # Assuming the third line contains a floating point number
                difficulty.append(float(third_line.strip()))  # Convert the line to a float
            else:
                # Handle case where file has less than 3 lines
                print("Metadata file doesn't have enough lines")
        problems.append(folder_name)  # problem name
        # description: taken from problem_text
        problem_text_dir = os.path.join(dir_path, folder_name, 'problem_text')
        if os.path.isfile(problem_text_dir):
            with open(problem_text_dir, 'r', encoding='utf-8') as problem_text_file:
                text = problem_text_file.read()
                description.append(text)
        else:
            print(f"Problem text file '{problem_text_dir}' not found.")
        completed_problem = completed_problem + 1


Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line
Could not extract numbers from the line


In [3]:
print(len(problems), len(difficulty), len(status), len(passCount), len(totalCount), len(description))

3323 3323 3323 3323 3323 3323


In [4]:
# dictionary of problem statistics
dict_full = {'Problem': problems,
             'Difficulty': difficulty,
             'Status': status,
             'Pass': passCount,
             'Total': totalCount,
             'Description': description} 
    
df_problems = pd.DataFrame(dict_full)
df_problems.to_csv('full_gpt_2.csv')